<a href="https://colab.research.google.com/github/elyesaseidel/motion-to-emotion/blob/main/Justin/Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install SpeechRecognition
!pip install streamlit -q
!pip install pyngrok
!pip install ffmpeg-python
!pip install python-settings
!pip install synthesizer
!pip install inference
!pip install helper
!pip install SpeechRecognition moviepy
!pip3 install imageio==2.4.1
!pip install imageio-ffmpeg
!pip install --upgrade moviepy
from pyngrok import ngrok
from google.colab import drive
from google.colab import files
import pandas as pd
import speech_recognition as sr
from IPython.display import HTML, Javascript
from google.colab import output
from base64 import b64decode
import base64
from google.colab.output import eval_js

# https://ngrok.com/ to sign in for free to get auth_key 
# Tutorial to run streamlit on colab
# https://www.youtube.com/watch?v=MUD-pBOnvdo 
ngrok.set_auth_token('2L2vPxTy5NQV7KFy7pIr8Vrm7M5_4YPqowvfnKA2fyQbtgktr')

# mount your drive with colab notebook
drive.mount('/content/gdrive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Testing - download and save file from/to google drive 

In [8]:
df = pd.read_csv('/content/gdrive/MyDrive/From Motion to Emotion/testing.csv')

In [9]:
df.head(2)

,Unnamed: 0,icao,iata,city_full_name,location,country_code
0,0,EDDT,TXL,"Berlin, Berlin-Tegel","{'lat': 52.5597, 'lon': 13.287699}",DE
1,1,EDDT,TXL,"Berlin, Berlin-Tegel","{'lat': 52.5597, 'lon': 13.287699}",DE


In [10]:
df_alt = df[:3]

In [11]:
path = '/content/gdrive/MyDrive/From Motion to Emotion/output.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df_alt.to_csv(f)

# Streamlit code

In [116]:
%%writefile Motion_to_Emotion.py 
import streamlit as st 
from pyngrok import ngrok
from google.colab import drive
from google.colab import files
from google.colab import output
import pandas as pd
import speech_recognition as sr
from IPython.display import Javascript
import base64
from base64 import b64decode
from google.colab.output import eval_js
import cv2
from deepface import DeepFace 
from collections import Counter
from pathlib import Path
import os
import glob
from collections import Counter
from PIL import Image
import moviepy.editor as mp
r = sr.Recognizer()

image = Image.open('/content/gdrive/MyDrive/From Motion to Emotion/emotion_imgage.jpeg')
st.image(image)#, caption='Sunrise by the mountains')

st.title('Motion to Emotion')




option = st.selectbox(
    'Select a video to analyse',
    ('Greta Thunberg\'s speech at UN', 'option2', 'option3'))

st.write('You selected:', option)

if option == 'Greta Thunberg\'s speech at UN':
  st.subheader('Greta Thunberg\'s speech at UN')
  st.video('/content/gdrive/MyDrive/From Motion to Emotion/Greta_UN.mp4')
  if st.button('Start Analysis'):


# Speech Recognition 
    clip = mp.VideoFileClip('/content/gdrive/MyDrive/From Motion to Emotion/Greta_UN.mp4') 
    clip.audio.write_audiofile('Greta_UN.wav')

    audio = sr.AudioFile("Greta_UN.wav")

    with audio as source:
      audio_file = r.record(source)
      result = r.recognize_google(audio_file)



# Face emotion dectection 
    # Extrac frames from video 
    video = cv2.VideoCapture('/content/gdrive/MyDrive/From Motion to Emotion/Greta_UN.mp4')
        
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    step = 30 # if step = 10, then only every 10-th frame will be considered and saved

    from collections import Counter
    count = 0

    frame_list = [] 

    for i in range(total_frames):
        ret, frame = video.read()
        if ret:
            count += 1
            if count % step == 0:
                frame_list.append(frame)
                
    video.release()

    # Analysis emotion from frames

    df = pd.DataFrame()
    emotions_list = []

    for picture in frame_list:

        objs = DeepFace.analyze(picture, actions = 'emotion', enforce_detection = False)
        dominant_emotion = objs[0]['dominant_emotion']
        emotions_list.append(dominant_emotion)

    # Create Output
    df['Emotion'] = Counter(emotions_list).keys()
    df['Counts'] = Counter(emotions_list).values()
    df['Percentage'] = (df['Counts']/df['Counts'].sum())*100
    
#    st.markdown('Face Emotion Detection during the video')
#    st.dataframe(df)
    
#    st.markdown('Most Domiant Emotion during the video')
#    st.dataframe(df[df.Percentage == df.Percentage.max()])

    percent = df.Percentage.max().round(2)

    df_sorted = df[['Emotion','Percentage']].sort_values(by = 'Percentage', ascending=False)
    df_sorted.reset_index(inplace=True)



# Result Columns 

    with st.container():
      st.header('Speech Recognition')
      st.write(result)

    with st.container():
      st.header("Face Emotion")
      col1, col2, col3 = st.columns(3)
      
      with col1:
        st.write('Most Deminant Face Emotion')
        if df_sorted.Emotion[0] == 'fear':
          st.title(':anguished:')
          st.subheader('Fear')
          st.header(percent)

      with col2:
        st.write('Second Deminant Face Emotion')
        if df_sorted.Emotion[0] == 'fear':
          st.title(':anguished:')
          st.subheader('Fear')
          st.header(percent)      

      with col3:
        st.write('Second Deminant Face Emotion')
        if df_sorted.Emotion[0] == 'fear':
          st.title(':anguished:')
          st.subheader('Fear')
          st.header(percent)     

Overwriting Motion_to_Emotion.py


# New Section

# Get Ngrok tunnel and connect

In [3]:
# get tunnel 
!nohup streamlit run Motion_to_Emotion.py --server.port 80 &
url = ngrok.connect(port = '80')
print(url)

nohup: appending output to 'nohup.out'
NgrokTunnel: "http://1122-34-80-44-56.ngrok.io" -> "http://localhost:80"


# Audio recording 

In [ ]:
js = Javascript("""
  async function recordAudio(){
    const div = document.createElement('div');
    const audio = document.createElement('audio');
    const strtButton = document.createElement('button');
    const stopButton = document.createElement('button');

    strtButton.textContent = 'Start Recording';
    stopButton.textContent = 'Stop Recording';

    document.body.appendChild(div);
    div.appendChild(strtButton);
    div.appendChild(audio);

    const stream = await navigator.mediaDevices.getUserMedia({audio:true});
    let recorder = new MediaRecorder(stream);

    audio.style.display = 'block';
    audio.srcObject = stream;
    audio.controls = true;
    audio.muted = true;

    await new Promise((resolve) => strtButton.onclick = resolve);
      strtButton.replaceWith(stopButton);
      recorder.start();

    await new Promise((resolve) => stopButton.onclick = resolve);
      recorder.stop();
      let recData = await new Promise ((resolve) => recorder.ondataavailable = resolve);
      let arrBuff = await recData.data.arrayBuffer();
      stream.getAudioTracks()[0].stop();
      div.remove()
      
      let binaryString = '';
      let bytes = new Uint8Array(arrBuff);
      bytes.forEach((byte) => { binaryString += String.fromCharCode(byte)});

    const url = URL.createObjectURL(recData.data);
    const player = document.createElement('audio');
    player.controls = true;
    player.src = url;
    document.body.appendChild(player);
  
  return btoa(binaryString)};
  """)

In [ ]:
# Record audio
display(js)
output = eval_js('recordAudio({})')
with open ('audio.wav','wb') as file:
    binary = base64.b64decode(output)
    file.write(binary)
print('Recording saved to:', file.name)

<IPython.core.display.Javascript object>

Recording saved to: audio.wav


# DeepFace Emotion Detection

In [ ]:
!pip install deepface
import cv2

In [7]:
# Open the video file from a folder 
video = cv2.VideoCapture('/content/gdrive/MyDrive/From Motion to Emotion/Greta_UN.mp4')
    
# Get the total number of frames in the video
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

# Set the step
step = 10 # if step = 10, then only every 10-th frame will be considered and saved

# Initialize the counter
from collections import Counter
count = 0

frame_list = [] 
# Iterate over each frame
for i in range(total_frames):
    ret, frame = video.read()
    if ret:
        count += 1
        if count % step == 0:
            frame_list.append(frame)
            #cv2.imwrite('/content/gdrive/MyDrive/From Motion to Emotion/picture/frame{}.jpg'.format(i), frame)
video.release()

In [ ]:
frame_list

In [ ]:
# from IPython.display import JSON #uncomment in case of an error
from deepface import DeepFace # to work with faces
from collections import Counter
from pathlib import Path
import os
import glob
from collections import Counter

df = pd.DataFrame()
list_frames_emotions = []

# video_frames_path = Path('/content/gdrive/MyDrive/From Motion to Emotion/picture')

# os.chdir(video_frames_path)

# video_frames = glob.glob("./*")

for picture in frame_list:

    objs = DeepFace.analyze(picture, actions = 'emotion', enforce_detection = False)
    dominant_emotion = objs[0]['dominant_emotion']
    list_frames_emotions.append(dominant_emotion)

df['Emotion'] = Counter(list_frames_emotions).keys()
df['Counts'] = Counter(list_frames_emotions).values()
df['Percentage'] = (df['Counts']/df['Counts'].sum())*100
print(df)
print(df[df.Percentage == df.Percentage.max()])


In [10]:
# most domiant emotion of the face expression
print(df[df.Percentage == df.Percentage.max()])

  Emotion  Counts  Percentage
1    fear      59   40.689655


## Video-Frame-DeepFace-Output (One Code)

In [ ]:
# !pip install deepface
import cv2
from deepface import DeepFace 
from collections import Counter
from pathlib import Path
import os
import glob
from collections import Counter

# Extrac frames from video 
video = cv2.VideoCapture('/content/gdrive/MyDrive/From Motion to Emotion/Greta_UN.mp4')
    
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

step = 10 # if step = 10, then only every 10-th frame will be considered and saved

from collections import Counter
count = 0

frame_list = [] 

for i in range(total_frames):
    ret, frame = video.read()
    if ret:
        count += 1
        if count % step == 0:
            frame_list.append(frame)
            
video.release()

# Analysis emotion from frames

df = pd.DataFrame()
emotions_list = []

for picture in frame_list:

    objs = DeepFace.analyze(picture, actions = 'emotion', enforce_detection = False)
    dominant_emotion = objs[0]['dominant_emotion']
    emotions_list.append(dominant_emotion)

# Create Output
df['Emotion'] = Counter(emotions_list).keys()
df['Counts'] = Counter(emotions_list).values()
df['Percentage'] = (df['Counts']/df['Counts'].sum())*100
print(df)
print(df[df.Percentage == df.Percentage.max()])


In [98]:
df

,Emotion,Counts,Percentage
0,neutral,15,10.344828
1,fear,59,40.689655
2,angry,28,19.310345
3,sad,22,15.172414
4,happy,12,8.275862
5,surprise,7,4.827586
6,disgust,2,1.379310


In [109]:
percent = df.Percentage.max().round(2)

In [110]:
print (percent, '%')

40.69 %


In [60]:
df_sorted = df[['Emotion','Percentage']].sort_values(by = 'Percentage', ascending=False)

In [64]:
df_sorted.reset_index(inplace=True)

In [86]:
# df_sorted[['Emotion','Percentage']] 
df_sorted.iloc[[1]]

,index,Emotion,Percentage
1,2,angry,19.310345


In [72]:
if df_sorted.Emotion[1] == 'fear':
  print('yes')
  
else:
    print('no')

no


In [ ]:
      # with col1:
      #   st.write('Most Deminant Face Emotion')
      #   if df_sorted.Emotion[0] == 'fear':
      #     st.title(':anguished:')
      #     st.subheader('Fear')
      #     st.header(percent)


In [119]:
df_sorted.Emotion[1]


'angry'

In [139]:
df_sorted[['Emotion','Percentage']][:1]

,Emotion,Percentage
0,fear,40.689655


In [123]:
if i in len(df_sorted.Emotion):
  df_sorted.Emotion[i] == 'fear'
  print('fear')
elif df_sorted.Emotion[i] == 'angry':
  print('angry')  

TypeError: ignored

# Extract Text from Video

In [12]:
# !pip install imageio-ffmpeg
import speech_recognition as sr
from moviepy.audio.AudioClip import AudioClip
from moviepy.audio.io.readers import FFMPEG_AudioReader

ModuleNotFoundError: ignored

In [40]:
audioclip = AudioFileClip('/content/gdrive/MyDrive/From Motion to Emotion/Greta_UN.mp4')
# audioclip.write_audiofile(f"audio_from_video_sm.wav")
# audio_to_be_text = (f"audio_from_video_sm.wav")

NameError: ignored

In [ ]:
r = sr.Recognizer()

# open the file
with sr.AudioFile(audio_to_be_text) as source:
    # listen for the data (load audio to memory)
    audio_data = r.record(source)
    # recognize (convert from speech to text)
    text = r.recognize_google(audio_data)
    print(text)

## new approach

In [4]:
!pip install SpeechRecognition moviepy
!pip3 install imageio==2.4.1
!pip install imageio-ffmpeg
!pip install --upgrade moviepy
import speech_recognition as sr 
import moviepy.editor as mp
r = sr.Recognizer()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached imageio-2.4.1-py3-none-any.whl
  Attempting uninstall: imageio
    Found existing installation: imageio 2.25.0
    Uninstalling imageio-2.25.0:
      Successfully uninstalled imageio-2.25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
moviepy 1.0.3 requires imageio<3.0,>=2.5; python_version >= "3.4", but you have imageio 2.4.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached imageio-2.25.0-py3-none-any.whl (3.4 MB)
  Attempting uninstall: imageio
    Found existing 

In [9]:
clip = mp.VideoFileClip('/content/gdrive/MyDrive/From Motion to Emotion/Greta_UN.mp4') 
clip.audio.write_audiofile('Greta_UN.wav')

audio = sr.AudioFile("Greta_UN.wav")

with audio as source:
  audio_file = r.record(source)
  result = r.recognize_google(audio_file)

result

MoviePy - Writing audio in Greta_UN.wav


MoviePy - Done.
result2:
{   'alternative': [   {   'confidence': 0.95733714,
                           'transcript': "this is all wrong I shouldn't be up "
                                         'here I should be back in school on '
                                         'the other side of the ocean yet you '
                                         'all come to us young people for Hope '
                                         'how dare you you have stolen my '
                                         'dreams and my childhood with your '
                                         "empty words and yet I'm one of the "
                                         'lucky ones people are suffering '
                                         'people are dying entire ecosystems '
                                         'are collapsing we are in the '
                                         'beginning of a mass extinction and '
                                         'all you can talk abo

"this is all wrong I shouldn't be up here I should be back in school on the other side of the ocean yet you all come to us young people for Hope how dare you you have stolen my dreams and my childhood with your empty words and yet I'm one of the lucky ones people are suffering people are dying entire ecosystems are collapsing we are in the beginning of a mass extinction and all you can talk about is the money and fairy tales of Eternal economic growth how dare you"

In [15]:
audio = sr.AudioFile("conveGreta_UNrted.wav")

In [ ]:
with audio as source:
  audio_file = r.record(source)
  result = r.recognize_google(audio_file)

In [17]:
result

"this is all wrong I shouldn't be up here I should be back in school on the other side of the ocean yet you all come to us young people for Hope how dare you you have stolen my dreams and my childhood with your empty words and yet I'm one of the lucky ones people are suffering people are dying entire ecosystems are collapsing we are in the beginning of a mass extinction and all you can talk about is the money and fairy tales of Eternal economic growth how dare you"

In [14]:
# exporting the result 
with open('recognized.txt',mode ='w') as file: 
   file.write("Recognized Speech:") 
   file.write("\n") 
   file.write(result) 
   print("ready!")

ready!


## 2 new approach

In [7]:
from tkinter import *
import moviepy.editor as mp 
import speech_recognition as sr


root=Tk()
root.geometry('750x420+480+0')
root.resizable(False,False)
root.config(bg='dark slate gray')
root.title('Video To Text Converter')

def convert():
    clip = mp.VideoFileClip(entry_vid.get()) 
    clip.audio.write_audiofile(entry_file.get()) 
        
def text():
    r = sr.Recognizer()
    audio = sr.AudioFile(entry_file.get())
    with audio as source:        
        audtext = r.listen(source)
        try:            
            text1 = r.recognize_google(audtext)
            st1.set(text1)
        except:
            st1.set('Try Again!')
   
def EXIT():
    root.destroy()
     
    


# Ltitle = Label(root,text='Video to Text Converter',font='Courier 16 underline bold',bg='dark slate gray',fg='mint cream').grid(row=0,column=2,ipadx=70)

# Lvid = Label(root,text="Video Path :",font='Constantia 14',bg='dark slate gray',fg='mint cream').grid(row=2,column=1,ipady=24,ipadx=18)
# entry_vid = Entry(root,width=70)
# entry_vid.insert(0,"FILE PATH .mp4")
# entry_vid.place(x=150,y=60)
# btn_con = Button(root,text='Convert to Text',font='Constantia 10',fg='maroon',command=convert).place(x=600,y=55)

# Lfile = Label(root,text='File Name :',font='Constantia 14',bg='dark slate gray',fg='mint cream').grid(row=3,column=1,ipady=8,ipadx=18)
# entry_file = Entry(root,width=70)
# entry_file.insert(0,"FILE NAME .wav")
# entry_file.place(x=150,y=120)

# st1 =StringVar()
# Ltext = Label(root,text="Text :",font='Constantia 14',bg='dark slate gray',fg='mint cream').grid(row=4,column=1)
# entry_text = Entry(root,width=70,textvariable=st1).grid(row=4,column=2,ipady=85)
# btn_text = Button(root,text='Get Text',font='Constantia 10',command=text,fg='maroon').place(x=300,y=360)


# butexit = Button(root,text= "Exit",command=EXIT,font='Constantia 10',fg='maroon').place(x=380,y=360)


# root.mainloop()

               
  

TclError: ignored